<a href="https://colab.research.google.com/github/SivapriyaVenkateswarar/BookIntel/blob/main/Data_pre_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
from datasets import load_dataset
ds = load_dataset("deepmind/narrativeqa")

Resolving data files:   0%|          | 0/24 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/24 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/18 [00:00<?, ?it/s]

In [ ]:
import shutil
total, used, free = shutil.disk_usage("/")
print(f"Disk Space - Total: {total // (2**30)} GB, Free: {free // (2**30)} GB")


Disk Space - Total: 107 GB, Free: 60 GB


In [ ]:
from datasets import concatenate_datasets
merged_dataset = concatenate_datasets([ds["train"], ds["test"], ds["validation"]])
print(f"Total samples: {len(merged_dataset)}")

Total samples: 46765


In [ ]:
import re

# Function to clean text
def clean_text(text):
    if not isinstance(text, str):  # Handle missing values
        return ""
    text = text.lower().strip()  # Lowercase and remove leading/trailing spaces
    text = re.sub(r"\s+", " ", text)  # Replace multiple spaces with a single space
    text = re.sub(r"[^a-zA-Z0-9?.!,;:()'\"\s]", "", text)  # Remove special characters
    return text

# Apply cleaning only to 'document' text
def preprocess_batch(batch):
    return {
        "document": [clean_text(doc.get("text", "")) for doc in batch["document"]],
        "question": [q.get("text", "") for q in batch["question"]],
        "answers": [[ans.get("text", "") for ans in a] for a in batch["answers"]]
    }

# Process dataset in batches
merged_dataset = merged_dataset.map(preprocess_batch, batched=True, batch_size=100, keep_in_memory=False)


Map:   0%|          | 0/46765 [00:00<?, ? examples/s]